In [1]:
import zeep
import numpy as np
import math
import time
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import estimator
from tensorflow import keras
from tensorflow.keras.utils import *
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split

from keras import backend as K

# import tensorflow_probability as tfp

In [6]:
client = zeep.Client('http://turbulence.pha.jhu.edu/service/turbulence.asmx?WSDL')
ArrayOfFloat = client.get_type('ns0:ArrayOfFloat')
ArrayOfArrayOfFloat = client.get_type('ns0:ArrayOfArrayOfFloat')
SpatialInterpolation = client.get_type('ns0:SpatialInterpolation')
TemporalInterpolation = client.get_type('ns0:TemporalInterpolation')

token = "uk.ac.imperial.george.dixon-dray17-29581e35"

In [15]:
def get_points_test(filter_width_n=9,origin_x=0,sample_nx=64,field="u",t=0):
    start_time = time.time()
    dataset = "isotropic1024coarse"

    # Database Domain Size
    domain_nx = 1024
    domain_x_min = 0
    domain_x_max = 2*np.pi
    domain_spacing = domain_x_max / (domain_nx)


    # Sample Domain Size
    filter_spacing = filter_width_n * domain_spacing
    print(filter_spacing)
    sample_x_min = math.ceil(filter_width_n/2) * domain_spacing + origin_x
    sample_x_max = sample_x_min + ((sample_nx-1) * filter_spacing)

    print("filter_spacing:", filter_spacing)

    x = np.linspace(sample_x_min, sample_x_max, sample_nx)
    print("x[1]-x[0]:", x[1]-x[0])
    y = x
    z = x
    X,Y,Z = np.meshgrid(x,y,z)
    X = X.ravel()
    Y = Y.ravel()
    Z = Z.ravel()

    nnp=5 #number of points
    points=np.random.rand(nnp,3)

    points = np.zeros((sample_nx**3,3))
    points[:,0] = Z
    points[:,1] = X
    points[:,2] = Y


    # convert to JHTDB structures
    x_coor=ArrayOfFloat(points[:,0].tolist())
    y_coor=ArrayOfFloat(points[:,1].tolist())
    z_coor=ArrayOfFloat(points[:,2].tolist())
    point=ArrayOfArrayOfFloat([x_coor,y_coor,z_coor]);
    
    return points

In [24]:
cube1_grad = get_points_test(filter_width_n=9,origin_x=0,sample_nx=8,field="u",t=0)
cube2_grad = get_points_test(filter_width_n=9,origin_x=np.pi,sample_nx=8,field="u",t=2)
cube3_grad = get_points_test(filter_width_n=9,origin_x=np.pi/2,sample_nx=8,field="u",t=2)
print(cube1_grad)
print(cube2_grad)
print(cube3_grad)

0.05522330836388308
filter_spacing: 0.05522330836388308
x[1]-x[0]: 0.05522330836388308
0.05522330836388308
filter_spacing: 0.05522330836388308
x[1]-x[0]: 0.05522330836388312
0.05522330836388308
filter_spacing: 0.05522330836388308
x[1]-x[0]: 0.05522330836388312
[[0.03067962 0.03067962 0.03067962]
 [0.08590292 0.03067962 0.03067962]
 [0.14112623 0.03067962 0.03067962]
 ...
 [0.30679616 0.41724277 0.41724277]
 [0.36201947 0.41724277 0.41724277]
 [0.41724277 0.41724277 0.41724277]]
[[3.17227227 3.17227227 3.17227227]
 [3.22749558 3.17227227 3.17227227]
 [3.28271889 3.17227227 3.17227227]
 ...
 [3.44838881 3.55883543 3.55883543]
 [3.50361212 3.55883543 3.55883543]
 [3.55883543 3.55883543 3.55883543]]
[[1.60147594 1.60147594 1.60147594]
 [1.65669925 1.60147594 1.60147594]
 [1.71192256 1.60147594 1.60147594]
 ...
 [1.87759248 1.9880391  1.9880391 ]
 [1.93281579 1.9880391  1.9880391 ]
 [1.9880391  1.9880391  1.9880391 ]]


In [25]:
x = np.concatenate((cube1_grad,cube2_grad,cube3_grad))
print(np.shape(x))

(1536, 3)


In [26]:
np.save('numpy_file_test', x)

In [29]:
y = np.load('numpy_file_test.npy')

In [30]:
print(np.shape(y))

(1536, 3)


In [52]:
from keras import backend as K
DNS = tf.constant(x)
print(DNS)
DNS_mean = tf.reduce_mean(DNS,0)
print(DNS_mean)
num_term_DNS = DNS - DNS_mean
print(num_term_DNS)

ANN = tf.constant(y)
ANN_mean = tf.reduce_mean(ANN,0)
num_term_ANN = ANN - ANN_mean
print(num_term_ANN)

num = tf.reduce_mean(tf.multiply(num_term_DNS,num_term_ANN),0)
print(num)

den_term_DNS = tf.sqrt(tf.reduce_mean(tf.square(num_term_DNS),0))
den_term_ANN = tf.sqrt(tf.reduce_mean(tf.square(num_term_ANN),0))
den = tf.multiply(den_term_DNS,den_term_ANN)
print(den)

r = tf.divide(num,den)
print(r)

tf.Tensor(
[[0.03067962 0.03067962 0.03067962]
 [0.08590292 0.03067962 0.03067962]
 [0.14112623 0.03067962 0.03067962]
 ...
 [1.87759248 1.9880391  1.9880391 ]
 [1.93281579 1.9880391  1.9880391 ]
 [1.9880391  1.9880391  1.9880391 ]], shape=(1536, 3), dtype=float64)
tf.Tensor([1.79475752 1.79475752 1.79475752], shape=(3,), dtype=float64)
tf.Tensor(
[[-1.76407791 -1.76407791 -1.76407791]
 [-1.7088546  -1.76407791 -1.76407791]
 [-1.65363129 -1.76407791 -1.76407791]
 ...
 [ 0.08283496  0.19328158  0.19328158]
 [ 0.13805827  0.19328158  0.19328158]
 [ 0.19328158  0.19328158  0.19328158]], shape=(1536, 3), dtype=float64)
tf.Tensor(
[[-1.76407791 -1.76407791 -1.76407791]
 [-1.7088546  -1.76407791 -1.76407791]
 [-1.65363129 -1.76407791 -1.76407791]
 ...
 [ 0.08283496  0.19328158  0.19328158]
 [ 0.13805827  0.19328158  0.19328158]
 [ 0.19328158  0.19328158  0.19328158]], shape=(1536, 3), dtype=float64)
tf.Tensor([1.66094454 1.66094454 1.66094454], shape=(3,), dtype=float64)
tf.Tensor([1.6609445

In [72]:
DNS = tf.constant(x)
ANN = tf.constant(y)
# N = K.int_shape(DNS)[0]

N = DNS.get_shape().as_list()[0]

num = N * tf.reduce_sum(tf.multiply(DNS,ANN),0) - tf.multiply(tf.reduce_sum(DNS,0),tf.reduce_sum(ANN,0))
print(num)
den_term_DNS = N * tf.reduce_sum(tf.square(DNS),0) - tf.square(tf.reduce_sum(DNS,0))
den_term_ANN = N * tf.reduce_sum(tf.square(ANN),0) - tf.square(tf.reduce_sum(ANN,0))

den = tf.sqrt(tf.multiply(den_term_DNS,den_term_ANN))
print(den)

r = tf.divide(num,den)
print(r)

# r_num = N * K.sum(tf.multiply(DNS,ANN)) - tf.multiply(K.sum(DNS),K.sum(ANN))
# r_den_term_DNS = N * K.sum(K.square(DNS)) - K.square(K.sum(DNS))
# r_den_term_ANN = N * K.sum(K.square(ANN)) - K.square(K.sum(ANN))
# r_den = K.sqrt(tf.multiply(r_den_term_DNS,r_den_term_ANN))

# r = tf.divide(r_num,r_den)

tf.Tensor([3918659.80762286 3918659.80762286 3918659.8076229 ], shape=(3,), dtype=float64)
tf.Tensor([3918659.80762286 3918659.80762286 3918659.8076229 ], shape=(3,), dtype=float64)
tf.Tensor([1. 1. 1.], shape=(3,), dtype=float64)


In [88]:
def correlation_coefficient_theo(x,y):
    DNS = x
    DNS_mean = tf.reduce_mean(DNS,0)
    num_term_DNS = DNS - DNS_mean
    
    ANN = y
    ANN_mean = tf.reduce_mean(ANN,0)
    num_term_ANN = ANN - ANN_mean
    
    num = tf.reduce_mean(tf.multiply(num_term_DNS,num_term_ANN),0)

    den_term_DNS = tf.sqrt(tf.reduce_mean(tf.square(num_term_DNS),0))
    den_term_ANN = tf.sqrt(tf.reduce_mean(tf.square(num_term_ANN),0))
    
    den = tf.multiply(den_term_DNS,den_term_ANN)

    r = tf.divide(num,den)
    return r

In [89]:
print(correlation_coefficient_theo(x,y))

tf.Tensor([1. 1. 1.], shape=(3,), dtype=float64)


In [131]:
def correlation_coefficient_alvaro(y_true,y_pred):
    DNS = y_true
    ANN = y_pred
    county_boi = tf.constant(DNS)
    N = tf.divide(tf.reduce_sum(DNS,0),tf.reduce_mean(DNS,0))
#     print(type(N[0]))
#     print(N[0].numpy())

    num = N * tf.reduce_sum(tf.multiply(DNS,ANN),0) - tf.multiply(tf.reduce_sum(DNS,0),tf.reduce_sum(ANN,0))
    
    den_term_DNS = N * tf.reduce_sum(tf.square(DNS),0) - tf.square(tf.reduce_sum(DNS,0))
    den_term_ANN = N * tf.reduce_sum(tf.square(ANN),0) - tf.square(tf.reduce_sum(ANN,0))
    den = tf.sqrt(tf.multiply(den_term_DNS,den_term_ANN))
    den_squared = tf.multiply(den_term_DNS,den_term_ANN)
#     tf.multiply(den_term_DNS,den_term_ANN)[0]
    r = tf.divide(num,den)
    den_sq_proto = tf.make_tensor_proto(den_squared)
    den_sq_np = tf.make_ndarray(den_sq_proto)
    den_sq0 = float(den_sq_np[0])
    print(type(den_sq_np))
    print(type(den_sq0))
    return tf.pow(tf.multiply(den_term_DNS,den_term_ANN),0.5)

def cc_comp_1(y_true,y_pred):
    r = correlation_coefficient_alvaro(y_true,y_pred)
    component = r[0]
    return component

In [132]:
print(correlation_coefficient_alvaro(x,y))
print(cc_comp_1(x,y))

<class 'numpy.ndarray'>
<class 'float'>
tf.Tensor([3918659.80762286 3918659.80762286 3918659.8076229 ], shape=(3,), dtype=float64)
<class 'numpy.ndarray'>
<class 'float'>
tf.Tensor(3918659.807622865, shape=(), dtype=float64)
